In [1]:
import os
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.utilities import GoogleSerperAPIWrapper
from fastapi import FastAPI
import requests
load_dotenv()

# 사전 세팅

from langchain_upstage import ChatUpstage
llm = ChatUpstage(model="solar-1-mini-chat")

def perform_company_search(user_input):
    # Initialize the Google Serper API wrapper
    search = GoogleSerperAPIWrapper(
        serper_parameters={
            "gl": "kr",           # Set geolocation to South Korea
            "hl": "ko",           # Set language to Korean
            "num": 50             # Number of results to retrieve (max 100)
        }
    )

    # Create the search query
    query = f"{user_input} 관련 매장 또는 회사 리스트"

    # Perform the search
    results = search.run(query)
    #print("res: ",results)
    
    return results

find_important_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 사용자의 관심사에서 핵심 주제나 관련된 회사를 추출하는 전문가입니다. 사용자의 입력에서 중요한 키워드나 회사 이름만을 추출하여 쉼표로 구분된 검색어 형태로 제공하세요. 만약 사용자 입력에 회사 이름이 직접 언급되었다면, 그 회사 이름들만 추출하세요. 그렇지 않으면, 핵심 주제나 분야를 가장 중요한 한 단어로 추출하세요."),
        ("human", "사용자 관심사: 11월에 진행되는 피잣집 할인 행사에 대해 알고 싶어"),
        ("ai", "피자"),
        ("human", "사용자 관심사: 이번주 놀이공원에 방문하려고 하는데, 어디가 좋을까?"),
        ("ai", "놀이공원"),
        ("human", "사용자 관심사: 피자헛이나 던킨 도넛에 대한 할인 행사가 지금 어떻게 돼"),
        ("ai", "피자헛, 던킨도넛"),
        ("human", "사용자 관심사: {user_input}"),
    ]
)

search_expansion_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 사용자의 관심사와 관련된 회사의 최근 프로모션, 할인 행사, 이벤트 정보를 찾기 위한 상세한 한국어 검색어를 생성하는 전문가입니다. 사용자의 입력과 관련된 회사 검색 결과를 기반으로, 해당 회사와 제품 또는 서비스에 맞는 현재 진행 중인 프로모션, 할인 행사, 특별 이벤트 등을 찾을 수 있는 구체적인 검색어를 생성하세요. 예를 들어 '롯데월드 최근 할인 행사', '에버랜드 최신 이벤트 정보', '파리바게뜨 신제품 출시 이벤트'와 같이 작성하세요. 구체적인 회사명을 '관련 회사 검색 결과'에서 찾아 반드시 포함시키세요. 문맥에 맞지 않는 부적절한 용어는 포함하지 마세요. 상시 또는 최신 이벤트 또는 프로모션임을 강조하세요. 결과는 쉼표로 구분된 한국어 검색어 리스트로 개수는 10개로 제공하세요."),
        ("human", "사용자 관심사: 놀이공원\n관련 회사 목록: 롯데월드, 에버랜드, 서울랜드, 경주월드, 광주패밀리랜드 ... 매장 이용관련 문의 및 불편사항 : 1577-1259 이메일 : elandfood@eland.co ... 회사 또는 제휴사와 관련하여 합병, 인수, 포괄적 영업양도 등이 있는 경우 합병 ... "),
        ("ai", "롯데월드 최신 할인 행사, 에버랜드 최신 이벤트 정보, 서울랜드 입장권 할인, 경주월드 특별 이벤트, 광주패밀리랜드 최근 프로모션"),
        ("human", "사용자 관심사: {user_input}\n관련 회사 검색 결과: {company_list}"),
    ]
)

pruned_search_expansion_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 사용자의 관심사에 기반하여 검색어 리스트에서 관련 없는 항목을 제거하는 전문가입니다. 사용자 관심사와 검색어 리스트가 주어지면, 사용자 관심사와 직접적으로 관련된 검색어만 남기고, 관련 없는 검색어는 제거하세요. 관련이 있는지 판별하기 위해 관련 회사 검색 결과를 참조하세요. 결과는 쉼표로 구분된 검색어 리스트로 제공하세요."),
        ("human", "사용자 관심사: 오늘 피자 땡기는데 피자 관련 행사 하는 거 있어?\n검색어 리스트: 미스터피자 할인 행사, 반올림피자 이벤트 정보, 피자스쿨 할인 메뉴, 피자마루 특별 이벤트, 피자헛 프로모션, 도미노피자 할인 행사, 피자알볼로 이벤트 정보, 59피자 할인 행사, 피자에땅 특별 이벤트, 피자마루 할인 행사, 배스킨라빈스 이달의 맛, 배스킨라빈스 할인 이벤트, 배스킨라빈스 이달의 할인, 배스킨라빈스 이 달의 쿠폰\n관련 회사 검색 결과: 매장수 1위: 피자스쿨과 피자마루 즉 저가 피자 take out 전문점이 ... 피자를 사랑하시는 여러분 또는 피자브랜드를 창업하고자 하시는 분들은 ... 이제 미스터피자에서 롯데카드 15% 할인! 기간 : 2017.11.08 ~ 2024.12.31. 온라인 방문포장 매장 식사 전화주문. 더보기. 미스터피자. 닫기. BC카드 인증 필수!! 확인 ... 정통 미국식 프리미엄 피자, 메뉴소개, 매장안내, 할 인이벤트, 가맹점 정보 안내. 반올림피자 ; 반올림X프라시아 전기 이벤트. 2024-10-28 ~ 2024-11-10 ; 매주 수요일은 반올림데 이. 2024-07-01 ~ 2024-11-30 ; 반올림의 선물. 2024-07-01 ~ 2024-11-30. 피자교실 진행 매장 ; 서울 · 서울대, 02-883-8486, 서울 관악구 관악로 246 (봉천동 37-86) ; 충북, 청주금천, 043-295-1576, 충청북도 청주시 상당구 금천동 287, 튼튼타워2 ... 2023년 신년 행사로 인기 메뉴 5종을 5,000원 할인 받을 수 있어 가성비 좋은 피자 맛집으로 입소문 나고 있다. 이곳에서는 자 연산 치즈 사용 및 저온 ... CREW쿠폰 사용가능 매장 리스트. 명동점, 세종로매장수 1위: 피자스쿨과 피자마루 즉 저가 피자 take out 전문점이 ... 피자를 사랑하시는 여러분 또는 피자브랜드를 창업하고자 하시는 분들은 ... 이제 미스터피자에서 롯데카드 15% 할인! 기간 : 2017.11.08 ~ 2024.12.31. 온라인 방문포장 매장식사 전화주문. 더보기. 미스터피자. 닫기. BC카드 인증 필수!! 확인 ... 정통 미국식 프리미엄 피자, 메뉴소개, 매장안내, 할인이벤트, 가맹점 정보 안내. 반올림피자 ; 반올 림X프라시아 전기 이벤트. 2024-10-28 ~ 2024-11-10 ; 매주 수요일은 반올림데이. 2024-07-01 ~ 2024-11-30 ; 반올림의 선물. 2024-07-01 ~ 2024-11-30. 피자교실 진행 매장 ; 서울 · 서울대, 02-883-8486, 서울 관악구 관악로 246 (봉천동 37-86) ; 충북, 청주금천, 043-295-1576, 충청북도 청주시 상당구 금천동 287, 튼튼타워2 ... 2023년 신년 행사로 인기 메뉴 5종을 5,000원 할인 받을 수 있어 가성비 좋은 피자 맛집 으로 입소문 나고 있다. 이곳에서는 자연산 치즈 사용 및 저온 ... CREW쿠폰 사용가능 매장 리스트. 명동점, 세종로점, 서대문점, 신촌점. 마포점, 망원점, 이문점, 면목점. 쌍문점, 옥수점, 천호암사점, 개포점. 공식 피자 주문 사이트, 온라인 전용 할인 메뉴, 매장, 제휴카드, 이벤트 소개, 대표번호 1588-5588. 피자를 주로 판매[2]하는 프랜차이즈 업체로 미국, 한국, 일본을 비롯하여 세계 곳곳에 진출해 있다. 매출과 매장수 기준 세계 1위이다. 한국에는 1990년 ... 일부매장(리조트 및 휴게소매장) 이용불가. OK 캐쉬백 관련 문의 : 고객센터 1599-0512. ※ 이용불가 매장안내 잠실야구장점, 알펜시아점, 대명비발디파크점, 롯데월드점"),
        ("ai", "미스터피자 할인 행사, 반올림피자 이벤트 정보, 피자스쿨 할인 메뉴, 피자마루 특별 이벤트, 피자헛 프로모션, 도미노피자 할인 행사, 피자알볼로 이벤트 정보, 59피자 할인 행사, 피자에땅 특별 이벤트, 피자마루 할인 행사"),
        ("human", "사용자 관심사: {user_input}\n검색어 리스트: {search_terms_list}\n관련 회사 검색 결과: {company_list}"),
    ]
)

# 시작
print("할인 행사 정보를 얻고 싶은 분야와 기간을 알려주세요.")
print("예시: 도넛 관련 할인 행사가 궁금해. 2024년 11월에 진행하는 거로.")
user_input = input("입력: ")

# 관련 분야 추출
chain_important = find_important_prompt | llm | StrOutputParser()
important_search_terms = chain_important.invoke({"user_input": user_input})
print("[System]사용자 입력 분석 중...")

# 관련 회사 검색
company_searched = perform_company_search(important_search_terms)

chain_vary = search_expansion_prompt | llm | StrOutputParser()
related_search_terms = chain_vary.invoke({"user_input": user_input, "company_list": company_searched})

#print(user_input, company_searched, related_search_terms,sep='\n')
# 적절하지 않은 검색어 가지치기
chain_vary_prune = pruned_search_expansion_prompt | llm | StrOutputParser()
related_search_terms = chain_vary_prune.invoke({"user_input": user_input, "search_terms_list": related_search_terms, "company_list": company_searched})
search_terms_list = [term.strip() for term in related_search_terms.split(",")]


print("[System]할인 행사 및 프로모션 검색 중...")


#############################

from bs4 import BeautifulSoup

def google_search(keyword):
    # 구글 검색 URL 생성
    url = f"https://www.google.com/search?q={keyword}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
    }
    
    # GET 요청 보내기
    response = requests.get(url, headers=headers)
    
    # 응답이 성공적이면
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 검색 결과에서 제목과 링크를 추출
        results = soup.find_all('div', class_='g')  # 각 검색 결과의 div 찾기
        if not results:
            return []
        
        search_results = []  # 결과를 저장할 리스트 초기화
        for result in results[0]:
            title = result.find('h3')
            link = result.find('a', href=True)  # href 속성을 가진 a 태그 찾기
            
            if title and link:
                search_results.append((title.text, link['href']))  # 제목과 URL을 튜플로 저장
                
        return search_results  # 결과 리스트 반환
    else:
        return []

# 전체 결과를 저장할 리스트 초기화
all_results = []

# 각 키워드에 대해 검색 결과 출력 및 저장
for keyword in search_terms_list:
    results = google_search(keyword)
    if results:
        all_results.append((keyword,results))

# 최종 결과 리스트 출력
print("[System]할인 행사 및 프로모션 요약 중...")
        
#####################

from openai import OpenAI  # OpenAI 라이브러리 임포트

# SoLar API 키 환경 변수에서 가져오기
solar_api_key = os.getenv("UPSTAGE_API_KEY")  # 적절한 환경 변수명으로 수정

def fetch_web_content(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers)
    except:
        return None
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.get_text()[:30000]
    else:
        return None



check_summary_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 제목과 정보가 모두 할인 프로모션과 관련된 정보가 포함되어 있는지, 정보가 이미 종료된 할인 프로모션에 대한 정보인지 확인하는 전문가입니다. 관련 정보가 존재하고 아직 할인을 제공중인 이벤트인 경우는 경우 O, 관련 정보가 없거나 이미 종료된 이벤트인 경우 X를 출력하세요."),
        ("human", "정보: 이번달 피자헛 할인 행사, 피자헛이 SKT T멤버십 고객 대상으로 프리미엄 피자를 50% 할인된 가격으로 제공하는 T day 프로모션을 진행하며, 방문 포장 주문 시 할인 혜택을 받을 수 있습니다. 신규 고객은 콜라 1.25L를 추가로  받을 수 있고, SKT의 구독 서비스인 우주패스에 가입하면 방문포장 전용 사이드팩 3종 쿠폰도 받을 수 있습니다."),
        ("ai", "O"),
        ("human", "제목: 이번달 기획, 정보: 제공된 텍스트에는 최대 3줄의 프로모션 정보가 포함되어 있지 않습니다."),
        ("ai", "X"),
        ("human", "제목: 종료 이벤트, 정보: 매그넘 코리아에서는 신년 이벤트를 진행하고 있습니다. 이벤트 상품으로는 매그넘17, 매그넘19, 남성용 방석 등이 있습니다. 이벤트에 대한 자세한 내용은 매그넘 코리아 웹사이트에서 확인할 수 있습니다."),
        ("ai", "X"),
        ("human", "제목: Exhibition - Digital Marketing Summit, 정보: DMS는 대한민국의 마케터들이 한 해의 방향성을 찾는 글로벌 지식 컨퍼런스입니다.\nDMS는 대한민국 마케터들이 한 해의 방향성을 찾는 글로벌 지식 컨퍼런스입니다. 이는 수백 개의 브랜드 기업의 마케터들이 참 가하는 행사이기도 합니다.\nDMS는 대한민국 마케터들이 한 해의 방향성을 찾는 글로벌 지식 컨퍼런스입니다. 이 컨퍼런스는 수백 개의 브랜드 기업의 마케 터들이 참가하는 행사이기도 합니다."),
        ("ai", "X"),
        ("human", "제목: {title}, 정보: {summary}"),
    ]
)

# LLM을 사용하여 텍스트 요약하기
def summarize_content(title, content):
    client = OpenAI(
        api_key=solar_api_key,
        base_url="https://api.upstage.ai/v1/solar"
    )

    # 프로모션 정보 추출 요청
    stream = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant that extracts promotion information. Please extract the main promotion information in max 3 lines. 최소 20자, 최대 200자 이내로 요약해 줘. 개인정보 보호 또는 서비스 이용 약관은 포함하면 안돼. If there is no promotion information, please print output with only 'X'."
            },
            {
                "role": "user",
                "content": f"Please extract the promotional information from the following text in max 3 lines.: \
                     파파존스 피자는 'Better Ingredients. Better Pizza'라는 슬로건을 내세워 품질과 맛에 대한 자신감을 표현하고 있습니다. 또한, 다양한 이벤트와 프로모션을 진행하여 고객들의 관심을 끌고 있습니다. 이 중에는 블루아카이브 프로모션, 아이브 포토카드 증정 이벤트, 파파스데이 이벤트, 초대형씰 경품 응모하기, 아이브 Pick! 기브투게더 세트, 블루멤버스 프로모션, 유독 Pick 프로모션, 파파 프라이데이, U+ 포인트 차감 결제 가능, 그린잇 식물성 피자 2종 판매, 신한 탑스클럽, 신규 회원님 쿠폰 증정, 더블 업그레이드 프로모션, 파파토크, 현대카드 결제 시 50% 포인트 차감 결제 가능, KB 국민카드 전용 세트 KB포인트리와 중복할인 가능 등이 있습니다.\
                    파파존스는 고객들의 개인정보를 중요하게 여기며, 개인정보 처리방침과 위치기반서비스 이용약관을 통해 개인정보 수집, 이용 목적, 보유기간, 개인정보 파기 절차 및 방법, 개인정보 공유 및 제공, 개인정보관리책임자 및 담당자의 연락처 등을 상세히 안내하고 있습니다. 또한, 고객의 개인정보 보호를 위해 최선을 다하고 있으며, 개인정보에 대한 문의 및 상담이 필요한 경우 개인정보침해신고센터, 대검찰청 사이버수사과, 경찰청 사이버안전국으로 문의할 수 있습니다.\
                    파파존스의 이용약관은 회원가입, 회원탈퇴, 고객정보 이용, 회원의 의무, 파파존스 서비스, 회원가입 시의 ID와 비밀번호, 파파존스 서비스 등에 대한 내용을 담고 있습니다. 파파존스는 사전 고지 없이 약관을 변경할 수 있으며, 변경된 약관은 초기 화면에 게재하거나 다른 방법으로 이용자에게 공지함으로써 효력을 발생합니다. 가입자가 변경된 약관에 동의하지 않는 경우 회원 등록을 취소할 수 있으며, 계속 사용의 경우는 약관 변경에 대한 동의로 간주됩니다."
            },
            {
                "role": "assistant",
                "content": "블루아카이브 프로모션, 아이브 포토카드 증정 이벤트, 파파스데이 이벤트, 초대형씰 경품 응모하기, 아이브 Pick! 기브투게더 세트, 블루멤버스 프로모션, 유독 Pick 프로모션, 파파 프라이데이, U+ 포인트 차감 결제 가능, 그린잇 식물성 피자 2종 판매, 신한 탑스클럽, 신규 회원님 쿠폰 증정, 더블 업그레이드 프로모션, 파파토크, 현대카드 결제 시 50% 포인트 차감 결제 가능, KB 국민카드 전용 세트 KB포인트리와 중복할인 가능 등이 있습니다."
            },
            {
                "role": "user",
                "content": f"Please extract the promotional information from the following text in max 3 lines.: \
                     Instagram에서 프로모션 정보를 찾을 수 없습니다."
            },
            {
                "role": "assistant",
                "content": "X"
            },
            {
                "role": "user",
                "content": f"Please extract the promotional information from the following text: {content}"
            }
        ],
        stream=False,  # 스트리밍 방식 비활성화
    )
    for i in stream.choices:
        summary = i.message.content
        chain_summary = check_summary_prompt | llm | StrOutputParser()
        summary_search_terms = chain_summary.invoke({"title":title, "summary": summary})
        if 'O' in summary_search_terms:
            return summary
    return ""


# 키워드 검색 결과 처리
idx=1
for keyword, results in all_results:
    out=[]
    for title, url in results:
        content = fetch_web_content(url)
        if content:
            summary = summarize_content(title, content)  # 요약 요청
            if len(summary)>10:
                out.extend([f"Title: {title}\nSummary: {summary}", f"출처: {url}"])
    if out:
        print(f"\n=========================\n")
        print(f"[#{idx}] {keyword}")
        for o in out:
            print(o)
        idx+=1
        if idx>10:
            break
            
print(f"\n=========================")
if idx==1:
    print("[System]원하는 정보를 찾을 수 없습니다. 다른 입력을 시도해주세요.")
else:
    print("[System]정보 제공이 완료되었습니다.")

할인 행사 정보를 얻고 싶은 분야와 기간을 알려주세요.
예시: 도넛 관련 할인 행사가 궁금해. 2024년 11월에 진행하는 거로.
[System]사용자 입력 분석 중...
[System]할인 행사 및 프로모션 검색 중...
[System]할인 행사 및 프로모션 요약 중...


[#1] 2024년 11월 국내 피자 할인 행사
Title: 피자마루, 11번가와 함께 최대 35% 할인 행사 진행
Summary: 피자 프랜차이즈인 피자마루가 11번가와 협력하여 '그랜드 십일절' 기념 프로모션을 진행합니다. 이 프로모션은 11월 1일부터 6일까지 진행되며, 매장 포장 주문 고객을 대상으로 최대 35% 할인 혜택을 제공합니다. 할인 대상 메뉴는 콤비네이션 피자 8,300원, 페퍼로니 피자 7,600원, 이탈리안 치즈 피자 6,170원입니다. 고객은 행사 기간 동안 11번가에서 피자마루를 검색하여 최대 35% 할인된 가격으로 쿠폰을 구매할 수 있습니다.
출처: http://www.shinailbo.co.kr/news/articleView.html?idxno=1953308


[#2] 2024년 11월 피자마루 할인 행사
Title: 피자마루, 11번가와 함께 최대 35% 할인 행사 진행
Summary: 피자마루는 11번가와 함께 최대 35% 할인 행사를 진행합니다. 이 행사는 고객들에게 감사의 마음을 전하고 기존 고객 유지 및 신규 고객 유치를 위해 마련되었습니다. 11월 1일부터 6일까지 진행되는 이 행사에서는 매장 포장 주문을 통해 피자를 주문하는 고객들은 최대 35% 할인 혜택을 받을 수 있습니다. 11번가에서 피자마루를 검색하면 쿠폰을 구매할 수 있습니다.
출처: https://www.shinailbo.co.kr/news/articleView.html?idxno=1953308


[#3] 2024년 11월 도미노피자 할인 행사
Title: 10월 SKT 0 day-이벤트
Summary: 10월 SKT 0 day 이벤트는 2024년 10월 30일부터 11월 3일까지